In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import nltk
from nltk.tag import pos_tag

In [2]:
url = 'https://www.nytimes.com/2017/02/20/us/politics/donald-trump-russia.html'

In [12]:
def find(paragraph, ch='"'):
    idxs =  [i for i, ltr in enumerate(paragraph) if ltr == ch]
    start, stop = idxs[0::2], idxs[1::2]
    quotes = [paragraph[idx: stop[i] + 1] for i, idx in enumerate(start)]
    if len(quotes) > 0:
        return quotes

def new_yorker(url):
    
    resp = requests.get(url=url)
    soup = BeautifulSoup(resp.content, 'lxml')

    body = soup.find('div', {'id' : 'articleBody'})
    text = ''

    for p in body.find_all('p'):
        text+=p.text 
    return text

def new_york_times(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    
#     body = soup.find('article', {'id' : 'story'}).find_all('p')
    body = soup.find('article', {'id' : 'story'}).find_all('p', {'class' : 'story-body-text story-content'})
    text = ''''''
    quotes = []
    
    for paragraph in body:
        paragraph_text = paragraph.text.replace('“', '"').replace('”', '"').replace('."', '." ')
        text += paragraph_text
        results = find(paragraph_text)
        
        if results:
            quotes.extend(results)
        
    
    return text, quotes

article, quotes = new_york_times(url)

In [13]:
quotes

['"I have nothing to do with Russia,"',
 '"To the best of my knowledge, no person that I deal with does."',
 '"immediate entourage"',
 '"I cannot say that all, but a number of them maintained contacts with Russian representatives,"',
 '"It’s something all diplomats do,"',
 '"This is a nonstory because to the best of our knowledge, no contacts took place, so it’s hard to make a comment on something that never happened,"',
 '"How many times do I have to answer this question? Russia is a ruse. I have nothing to do with Russia,"',
 '"illegal"',
 '"They want to better understand policy views of a particular candidate so they can perhaps make their case for certain policies,"',
 '"it’s about relationship building — they want to get to know the people who are possibly going to be in important jobs,"',
 '"everyone in Moscow knew that I was advising the campaign."',
 '"I was not there to discuss Obama policy but to better inform my views on Russian attitudes about U.S.-Russia relations,"',
 '"o

In [20]:
words_df[(words_df.Title == True) & (words_df.Kind.str.contains('NN'))]

,Words,Title,Kind
4,President,True,NNP
5,Trump,True,NN
25,Mr.,True,NNP
26,Trump,True,NN
35,"Russia,",True,NN
40,Thursday.,True,NN
74,Mr.,True,NNP
75,Trump,True,NN
136,Nov.,True,NNP
140,Sergei,True,NN


In [21]:
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
    

body = soup.find('article', {'id' : 'story'}).find_all('p', {'class' : 'story-body-text story-content'})


In [22]:
soup.find('article', {'id' : 'story'}).find_all('a')

[<a href="https://www.nytimes.com/pages/politics/index.html">Politics</a>,
 <a href="https://www.nytimes.com/by/matthew-rosenberg" title="More Articles by MATTHEW ROSENBERG"><span class="byline-author" data-byline-name="MATTHEW ROSENBERG" data-twitter-handle="AllMattNYT" itemprop="name">MATTHEW ROSENBERG</span></a>,
 <a class="visually-hidden skip-to-text-link" href="#story-continues-1">Continue reading the main story</a>,
 <a class="visually-hidden skip-to-text-link" href="#story-continues-1">Continue reading the main story</a>,
 <a href="https://www.nytimes.com/2017/02/16/us/politics/trump-news-conference.html">told reporters on Thursday</a>,
 <a class="visually-hidden skip-to-text-link" href="#story-continues-1">Continue reading the main story</a>,
 <a href="https://www.nytimes.com/2016/11/11/world/europe/trump-campaign-russia.html">said his </a>,
 <a href="https://www.nytimes.com/2016/11/11/world/europe/trump-campaign-russia.html"> government had maintain</a>,
 <a href="https://www

#### There are several types of quotes in articles:
    1. Full sentence quotes
    2. Snippet quotes
    3. Two part quotes
    4. One Word quotes

#### We are interested in full sentence and two part quotes. Try to find the name associated with the quote

In [256]:
article = article.replace('Mr.', 'Mr')
sentences = pd.Series(article.split('. '))
sentences[sentences.str.contains('"')][0]

'WASHINGTON — For months, President Trump and his aides have insisted that they had no contact with Russian officials during the presidential campaign, a denial Mr Trump repeated last week."I have nothing to do with Russia," he told reporters on Thursday'

In [17]:
def find_proper_nouns(x):
    try:
        return pos_tag([x])[0][1]
    except:
        return None

In [18]:
words = pd.Series(article.split(' '))
prop_noun = np.array(pos_tag(words[words.str.istitle()].str.replace('"', '').values))
prop_noun_group = pd.DataFrame(prop_noun, columns=['Words', 'Group'])
final_pn = prop_noun_group[prop_noun_group.Group.str.contains('NNP')]

In [19]:
words_df = pd.DataFrame(article.split(' '), columns=['Words']).applymap(lambda x: x.replace('"', ''))
words_df['Title'] = words_df.Words.apply(lambda x: x.istitle())
words_df['Kind'] = words_df.Words.apply(find_proper_nouns)
words_df[(words_df.Title == True) & (words_df.Kind.str.contains('NN'))]

,Words,Title,Kind
4,President,True,NNP
5,Trump,True,NN
25,Mr.,True,NNP
26,Trump,True,NN
35,"Russia,",True,NN
40,Thursday.,True,NN
74,Mr.,True,NNP
75,Trump,True,NN
136,Nov.,True,NNP
140,Sergei,True,NN


Find the indexes of all of the periods, consider the following:

    1. If the last word before the period is one letter, then it is an ititial
    2. If the word after the period is NOT title(), then it is not an actual sentence end

162

In [115]:
soup.find('article', {'id' : 'story'}).find_all('p', {'class' : 'story-body-text story-content'})

[<p class="story-body-text story-content" data-para-count="191" data-total-count="191">WASHINGTON — For months, President Trump and his aides have insisted that they had no contact with Russian officials during the presidential campaign, a denial Mr. Trump repeated last week.</p>,
 <p class="story-body-text story-content" data-para-count="138" data-total-count="329">“I have nothing to do with Russia,” he <a href="https://www.nytimes.com/2017/02/16/us/politics/trump-news-conference.html">told reporters on Thursday</a>. “To the best of my knowledge, no person that I deal with does.”</p>,
 <p class="story-body-text story-content" data-para-count="154" data-total-count="483">The denial stands at odds with statements by Russian officials, who have at least twice acknowledged contacts with aides to Mr. Trump before the election.</p>,
 <p class="story-body-text story-content" data-para-count="332" data-total-count="815">It is not uncommon for a presidential campaign to speak to foreign offici

In [ ]:
def find_quotes_and_speaker(article):
    # break out into sentences
    article = article.replace('Mr.', 'Mr')
    pd.Series(article.split('.'))